In [1]:
!pip install librosa
import os
import numpy as np
import librosa
import glob

In [2]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Path to the TESS dataset
data_path = 'TESS'

# Emotions in the dataset
emotions = ['OAF_angry', 'OAF_disgust', 'OAF_Fear', 'OAF_happy','OAF_neutral','OAF_Pleasant_surprise','OAF_sad','YAF_angry','YAF_disgust','YAF_fear','YAF_happy','YAF_neutral','YAF_pleasant_surprised','YAF_sad']

In [3]:
# Initialize lists to hold features and labels
X = []
y = []

# Iterate over each emotion and each file within each emotion directory
for emotion in emotions:
    files = glob.glob(os.path.join(data_path, f'*{emotion}/*.wav'))
    for file in files:
        features = extract_features(file)
        X.append(features)
        y.append(emotion)

X = np.array(X)
y = np.array(y)

In [4]:
X

array([[-3.9003134e+02,  5.5235966e+01, -1.5034650e+01, ...,
         1.6460081e+00, -9.4095045e-01,  2.2132027e+00],
       [-4.5101947e+02,  6.7048828e+01, -4.4825765e-01, ...,
        -7.1881580e+00,  3.7225678e+00,  4.8621292e+00],
       [-4.0656332e+02,  2.8609005e+01, -5.3172159e+00, ...,
        -1.1231725e+01,  2.0155830e+00,  3.0970709e+00],
       ...,
       [-3.7334085e+02,  6.6174812e+01,  3.6591633e+01, ...,
        -1.2031945e+01,  2.7324443e+00, -1.0008318e+01],
       [-4.2718451e+02,  7.0532188e+01,  2.8011265e+01, ...,
        -8.4890594e+00, -8.2178056e-01, -8.5601988e+00],
       [-4.1227975e+02,  5.3551804e+01,  3.1432125e+01, ...,
        -8.9182196e+00,  4.7999001e+00, -9.2423496e+00]], dtype=float32)

In [5]:
# Encode the labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [6]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau

In [7]:
# Create the model
def create_model(input_shape):
    model = Sequential()
    model.add(Dense(256, input_shape=(input_shape,), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(emotions), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Initialize and train the model
model = create_model(input_shape=X.shape[1])
model.fit(X, y_encoded, epochs=50, batch_size=32, validation_split=0.2)

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.0993 - loss: 18.5033 - val_accuracy: 0.0000e+00 - val_loss: 3.4349
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1078 - loss: 3.7991 - val_accuracy: 0.0000e+00 - val_loss: 2.8870
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1160 - loss: 2.7713 - val_accuracy: 0.0000e+00 - val_loss: 2.8068
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1467 - loss: 2.4587 - val_accuracy: 0.0000e+00 - val_loss: 3.4874
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1713 - loss: 2.2967 - val_accuracy: 0.0000e+00 - val_loss: 4.4341
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1995 - loss: 2.1553 - val_accuracy: 0.0000e+00 - val_loss: 5.5996
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1891 - loss: 2.0486 - val_accuracy: 0.0000e+00 - val_loss: 6.2505
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2256 - loss: 1.9636 - v

In [9]:
print("Accuracy of the model on the test data : " , model.evaluate(X,y_encoded)[1]*100,'%')

88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9403 - loss: 0.4547
Accuracy of the model on the test data :  80.46428561210632 %


In [10]:
#Save the model
model.save('emotion_recognition_model0.model.h5')